# Importing Libraries

In [ ]:
!pip install mne
!apt-get install libfftw3-dev
!pip install stockwell

     |████████████████████████████████| 6.9MB 5.0MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfftw3-bin libfftw3-long3 libfftw3-quad3 libfftw3-single3
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-long3 libfftw3-quad3 libfftw3-single3
0 upgraded, 5 newly installed, 0 to remove and 15 not upgraded.
Need to get 3,766 kB of archives.
After this operation, 21.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfftw3-long3 amd64 3.3.7-1 [308 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfftw3-quad3 amd64 3.3.7-1 [552 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfftw3-single3 amd64 3.3.7-1 [764 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfftw3-bin amd64 3.3.7-1 [32.7 kB]
Get:5 http://archive.ubuntu.co

In [ ]:
import sys
import os
import numpy as np
import mne
from mne.preprocessing import ICA
import warnings
import stockwell
import stockwell.smt as smt
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
def load_eeg_raw(subj_type, id):
    subject_folder = f'/content/drive/MyDrive/Mendeley_Database/{subj_type}_Subjects'
    subject_raw_file = os.path.join(subject_folder, f'{subj_type}_EDF', f'{subj_type}_Subject_{id}.edf')
    raw = mne.io.read_raw_edf(subject_raw_file, preload=True)
    return raw

# Preprocessing Dataset

In [ ]:
def fit_ica(raw, exclude, crop_time):
    raw.crop(tmax=crop_time)
    ica = ICA(n_components=0.9999, random_state=960, max_iter=200)
    ica.fit(raw)
    return raw, ica

def plot_ica_sources(raw, ica):
    ica.plot_sources(raw, show_scrollbars=False)
    del raw

def apply_ica(raw, ica):
    ica.apply(raw)
    return raw

# Annotating, transforming and plotting data

In [ ]:
def get_sleep_profile(subj_type, id):
    """Return sleep profile and its reliability"""    
    sleep_profile = []
    subject_folder = f'/content/drive/MyDrive/Mendeley_Database/{subj_type}_Subjects'
    subject_profile_path = os.path.join(subject_folder, f'{subj_type}_Outputs', 
                                         f'PSG_Output_{subj_type}_Subject_{id}', 'Sleep Profile.txt')
    subject_profile_path_rel = os.path.join(subject_folder, f'{subj_type}_Outputs', 
                                         f'PSG_Output_{subj_type}_Subject_{id}', 'Sleep Profile Reliability.txt')
    
    with open(subject_profile_path, 'rt') as profile, open(subject_profile_path_rel, 'rt') as profile_rel:
        for line_p, line_pr in zip(list(profile)[7:], list(profile_rel)[6:]):
            s_profile = []
            if (line_p.find(';') != -1 and line_pr.find(';') != -1):
                # Sleep profile
                line_p = line_p.rstrip('\n').split(';')
                line_p[1] = line_p[1].lstrip(' ')
                s_profile.append(line_p[1])
                # Sleep profile reliability
                line_pr = line_pr.rstrip('\n').split(';')
                line_pr[1] = line_pr[1].lstrip(' ')
                s_profile.append(line_pr[1].split(' ')[0])
        
            sleep_profile.append(s_profile)
    return sleep_profile

In [ ]:
def get_sleep_profile_2(subj_type, id):
    """Return sleep profile and its reliability"""    
    sleep_profile = []
    subject_folder = f'/content/drive/MyDrive/Mendeley_Database/{subj_type}_Subjects'

    subject_profile_path = os.path.join(subject_folder, 
                                        f'{subj_type}_Outputs', 
                                        f'PSG_Output_{subj_type}_Subject_{id}', 
                                        'Sleep Profile.txt')
    
    subject_profile_path_rel = os.path.join(subject_folder,
                                            f'{subj_type}_Outputs', 
                                            f'PSG_Output_{subj_type}_Subject_{id}', 
                                            'Sleep Profile Reliability.txt')
    
    with open(subject_profile_path, 'rt') as profile, open(subject_profile_path_rel, 'rt') as profile_rel:
        # Sleep profile
        for line in list(profile):
            if (line.find(';') != -1):
                line = line.rstrip('\n').split(';')
                line[1] = line[1].lstrip(' ')
                sleep_profile.append([line[1]])
        
        # Sleep profile reliability
        index = 0
        for line in list(profile_rel):
            if (line.find(';') != -1):
                line = line.rstrip('\n').split(';')
                line[1] = line[1].lstrip(' ')
                if (index < len(sleep_profile)):
                    sleep_profile[index].append(line[1])
                index += 1
        
        return sleep_profile

In [ ]:
def set_annotations_to_raw(raw, sleep_profile):
    """Read annotations from txt file and set it to raw data"""
    sleep_profile = np.array(sleep_profile)
    onset = np.arange(start=0, stop=30*len(sleep_profile), step=30)
    durations = [30] * len(sleep_profile)
    descriptions = sleep_profile[:, 0]
    normal_subject_01_annot = mne.Annotations(onset, durations, 
                                              descriptions, 
                                              orig_time=raw.info['meas_date'])
    raw.set_annotations(normal_subject_01_annot)
    return raw

In [ ]:
def plotspec(psx, fs, lofreq=None, hifreq=None, t1=None, t2=None):
    """Modify dimensions and properties of stockwell transformed image"""
    extent = [0, psx.shape[1], 0.0, fs/2.0]
    if (t1 != None and t2 != None):
        extent[0] = t1
        extent[1] = t2
    if (lofreq != None):
        extent[2] = lofreq
    if (hifreq != None):
        extent[3] = hifreq

    return plt.imshow(psx, extent=extent, aspect='auto', origin='lower')

In [ ]:
def stockwell_transform(segment):
    """Return stockwell transformed epoch"""
    return abs(smt.st(segment))

In [ ]:
def plot_epochs_stockwell(epochs, sleep_profile, sleep_type=None, 
                          id=None, save=False, high_reliability=False):
    """Plot stockwell transformed epochs"""
    sleep_profile = mask_profile(sleep_profile, epochs.drop_log)
    for rel, i in zip(sleep_profile, range(len(epochs))):
        if (rel[1] == '100%-70% Reliability'):
            segment = epochs[i].get_data().flatten()
            segment_st = stockwell_transform(segment)
            plt.yscale('log')
            fig = plotspec(segment_st, 300.0)
            plt.ylim(0, 50) # Cropping max frequency to 45Hz
            plt.axis('off')
            if (save):
                plt.savefig(f'''/content/drive/MyDrive/images/{rel[0]}/{
                            sleep_type}_{id}_{rel[0]}_{i}.png''', 
                            bbox_inches='tight', pad_inches = 0)            
            else:
                plt.title(f'{rel[0]}, {rel[1]}')
                plt.show()
            plt.clf()
            del segment_st 
            del fig
            # !cat /proc/meminfo

In [ ]:
def mask_profile(sleep_profile, mask):
    """Drop corresponding profiles according to dropped epochs"""
    import operator
    mask = list(map(operator.not_, list(map(bool, mask))))
    return np.array(sleep_profile)[mask]

In [ ]:
import time
def plot_epochs(epochs, sleep_profile):
    """Plots each 30 second epoch"""
    for rel, i in zip(sleep_profile, range(len(epochs))):
        if not (epochs.drop_log[i]):
            print((rel[0], rel[1]))
            epochs[i].plot(show_scrollbars=False)
            time.sleep(1)

In [ ]:
type_of_subj = ['Insomniac']
for t in type_of_subj:
    for id in range(6, 12):
        raw = load_eeg_raw(t, id)
        sleep_profile = get_sleep_profile_2(t, id)
        raw = set_annotations_to_raw(raw, sleep_profile)
        raw, ica = fit_ica(raw, exclude=[0], crop_time=None)
        plot_ica_sources(raw, ica) 
        apply_ica(raw, ica)
        raw.pick_channels(['O1'])
        events, event_id = mne.events_from_annotations(raw)
        epochs = mne.Epochs(raw, events, event_id, 
                            tmin=-0.1, tmax=30, preload=True)
        del raw, ica, events
        plot_epochs_stockwell(epochs, sleep_profile, t, id, save=True)
        del epochs

# Splitting images, training dataset

In [ ]:
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/drive/MyDrive/images', output="/content/output", seed=4269, ratio=(.8, 0.1, 0.1))

Copying files: 8069 files [1:04:34,  2.08 files/s]


In [ ]:
import os
# base_dir = '/content/output_binary/'
# base_dir = '/content/output_ternary/'
base_dir = '/content/output'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')
labels_to_values = {'Stage 1': 1, 'Stage 2': 2, 'Stage 3': 3, 'Wake': 4, 'Rem': 5}

labels_to_dir = {}
for l in labels_to_values:
    labels_to_dir[f'train_{l}_dir'] = os.path.join(train_dir, l)
    labels_to_dir[f'val_{l}_dir'] = os.path.join(validation_dir, l)
    labels_to_dir[f'test_{l}_dir'] = os.path.join(test_dir, l)
print(labels_to_dir)

{'train_Stage 1_dir': '/content/output/train/Stage 1', 'val_Stage 1_dir': '/content/output/val/Stage 1', 'test_Stage 1_dir': '/content/output/test/Stage 1', 'train_Stage 2_dir': '/content/output/train/Stage 2', 'val_Stage 2_dir': '/content/output/val/Stage 2', 'test_Stage 2_dir': '/content/output/test/Stage 2', 'train_Stage 3_dir': '/content/output/train/Stage 3', 'val_Stage 3_dir': '/content/output/val/Stage 3', 'test_Stage 3_dir': '/content/output/test/Stage 3', 'train_Wake_dir': '/content/output/train/Wake', 'val_Wake_dir': '/content/output/val/Wake', 'test_Wake_dir': '/content/output/test/Wake', 'train_Rem_dir': '/content/output/train/Rem', 'val_Rem_dir': '/content/output/val/Rem', 'test_Rem_dir': '/content/output/test/Rem'}


In [ ]:
for l in labels_to_values:
    print(f'total training {l} images:', len(os.listdir(labels_to_dir[f'train_{l}_dir'])))
    print(f'total validation {l} images:', len(os.listdir(labels_to_dir[f'val_{l}_dir'])))
    print(f'total test {l} images:', len(os.listdir(labels_to_dir[f'test_{l}_dir'])))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    # shear_range=0.2,
    # zoom_range=0.2,
    rescale=1./255)
val_datagen = ImageDataGenerator(
    # shear_range=0.2,
    # zoom_range=0.2,
    rescale=1./255)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    # shear_range=0.2,
    # zoom_range=0.2)
    )

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        class_mode='categorical',
        classes=['Stage_1_new', 'Stage_2_new', 'Stage_3_new', 'Wake_new', 'Rem'],
        target_size=(256, 256))

# Flow validation images in batches of 32 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        class_mode='categorical',
        classes=['Stage 1', 'Stage 2', 'Stage 3', 'Wake', 'Rem'],
        target_size=(256, 256))

test_generator = test_datagen.flow_from_directory(
        test_dir,
        shuffle=False,
        class_mode='categorical',
        classes=['Stage 1', 'Stage 2', 'Stage 3', 'Wake', 'Rem'],
        target_size=(256, 256))

Found 5140 images belonging to 5 classes.
Found 779 images belonging to 5 classes.
Found 786 images belonging to 5 classes.


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
img_input = layers.Input(shape=(256, 256, 3))

x = layers.Conv2D(10, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(20, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(30, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(40, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(50, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(5, activation='softmax')(x)

model = Model(img_input, output)
model.summary()
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics='accuracy')

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 254, 254, 10)      280       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 127, 127, 10)      0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 125, 125, 20)      1820      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 62, 62, 20)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 60, 60, 30)        5430      
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 30, 30, 30)        0   

In [ ]:
!zip -r /content/images.zip /content/images/
# from google.colab import files
# files.download('/content/output.zip') 

In [ ]:
!unzip /content/images.zip

In [ ]:
del model

In [ ]:
history = model.fit_generator(
      train_generator,
      epochs=50,
      validation_data=validation_generator,
      shuffle=False,
      verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
161/161 [==============================] - 26s 156ms/step - loss: 1.4012 - accuracy: 0.3690 - val_loss: 1.1336 - val_accuracy: 0.4647
Epoch 2/50
161/161 [==============================] - 25s 154ms/step - loss: 1.1212 - accuracy: 0.5189 - val_loss: 1.0966 - val_accuracy: 0.4596
Epoch 3/50
161/161 [==============================] - 25s 153ms/step - loss: 1.0466 - accuracy: 0.5527 - val_loss: 0.9899 - val_accuracy: 0.5687
Epoch 4/50
161/161 [==============================] - 25s 152ms/step - loss: 1.0022 - accuracy: 0.5793 - val_loss: 1.0215 - val_accuracy: 0.5866
Epoch 5/50
161/161 [==============================] - 25s 154ms/step - loss: 0.9536 - accuracy: 0.6090 - val_loss: 1.0561 - val_accuracy: 0.5173
Epoch 6/50
161/161 [==============================] - 25s 152ms/step - loss: 0.9017 - accuracy: 0.6279 - val_loss: 1.0111 - val_accuracy: 0.5533
Epoch 7/50
161/161 [==============================] - 25s 154ms/step - loss: 0.8360 - accuracy: 0.6562 - val_loss: 0.9234 - val_ac

KeyboardInterrupt: ignored

# Evaluation and Result Graphs

In [ ]:
#@title Stage 1 vs Stage 2 vs Stage 3 vs Wake vs Rem
import matplotlib.pyplot as plt

# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

model.evaluate(x=test_generator)
y_pred = (model.predict(test_generator))
y_pred = np.argmax(y_pred, axis=1)
cm=confusion_matrix(test_generator.classes,y_pred)
print(test_generator.class_indices)
print("Confusion matrix is")
print(cm)
for i in range(5):
  print("Accuracy for class ", i)
  print((cm.diagonal()/cm.sum(axis=1))[i] *100) #Individual Class Accuracy

25/25 [==============================] - 3s 126ms/step - loss: 1.2988 - accuracy: 0.5662
{'Stage 1': 0, 'Stage 2': 1, 'Stage 3': 2, 'Wake': 3, 'Rem': 4}
Confusion matrix is
[[125  28  29  40  70]
 [ 18 102  34   3   3]
 [  4  45 101   3   3]
 [ 10   5   4 106  16]
 [ 14   2   6   4  11]]
Accuracy for class  0
42.80821917808219
Accuracy for class  1
63.74999999999999
Accuracy for class  2
64.74358974358975
Accuracy for class  3
75.177304964539
Accuracy for class  4
29.72972972972973
